# APIM Costing & Showback Sample

This notebook deploys and configures resources to track and allocate API costs using Azure API Management.

📖 See [README.md](README.md) for detailed information about this sample.

## 🎯 What This Sample Does

1. Deploys observability stack (Application Insights, Log Analytics, Storage Account)
2. Configures APIM diagnostic settings to capture request logs
3. Creates sample API and multiple subscriptions representing different business units
4. Sets up Cost Management export for cost data
5. Deploys Azure Monitor Workbook for cost visualization
6. Generates sample API traffic to demonstrate cost tracking
7. Provides Kusto queries for cost analysis

## ⚙️ Initialize Notebook Variables

Configure the parameters for your environment.

❗️ **Modify entries under _User-defined parameters_**.

In [ ]:
import sys
import time
from pathlib import Path

# APIM Samples imports
import azure_resources as az
from apimtypes import INFRASTRUCTURE
from console import print_error, print_info, print_ok, print_val, print_warning
import utils

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

# Infrastructure to use (must be deployed first via infrastructure/simple-apim/create.ipynb)
infrastructure = INFRASTRUCTURE.SIMPLE_APIM  # Options: SIMPLE_APIM, APPGW_APIM, etc.
index = 4                                     # Infrastructure index (must match your deployed infrastructure)

# Sample deployment configuration
sample_index = 2                              # Sample deployment index (increment for multiple deployments)

# Cost export configuration
cost_export_frequency = 'Daily'               # Options: 'Daily', 'Weekly', 'Monthly'

# Sample data generation
generate_sample_load = True                   # Generate sample API calls to demonstrate cost tracking
sample_requests_per_subscription = 50         # Base request count per BU (multiplied by each BU's weight)

# Optional: Use your own existing APIM deployment (uncomment both lines below)
existing_rg_name = 'apim-infra-simple-apim-4'
existing_apim_name = 'apim-5lbrwbpu7nwii'



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

sample_folder = 'costing'

# Determine resource group name
if 'existing_rg_name' in dir() and 'existing_apim_name' in dir():
    rg_name = existing_rg_name
    apim_name = existing_apim_name
    print_info(f'Using existing APIM: {apim_name} in {rg_name}')
else:
    rg_name = az.get_infra_rg_name(infrastructure, index)
    apim_name = None  # Will be auto-detected in cell 6
    print_info(f'Using infrastructure: {infrastructure.value} (index: {index})')

# Check resource group exists
if not az.does_resource_group_exist(rg_name):
    print_error(f'Resource group "{rg_name}" does not exist.')
    print_info(f'Deploy infrastructure first: infrastructure/{infrastructure.value}/create.ipynb')
    raise SystemExit(1)

rg_location = az.get_resource_group_location(rg_name)

# Get Azure subscription ID
account_output = az.run('az account show --query id -o tsv', log_command=False)
subscription_id = account_output.text.strip() if account_output.success else None

if not subscription_id:
    print_error('Could not determine Azure subscription ID. Run: az login')
    raise SystemExit(1)

# Store configuration for later use
config = {
    'rg_name': rg_name,
    'apim_name': apim_name,
    'location': rg_location,
    'subscription_id': subscription_id,
    'cost_export_frequency': cost_export_frequency,
    'generate_sample_load': generate_sample_load,
    'sample_requests_per_subscription': sample_requests_per_subscription,
    'sample_folder': sample_folder,
    'infrastructure': infrastructure,
    'index': index,
    'sample_index': sample_index
}

print_ok('Configuration loaded')
print_val('Resource Group', config['rg_name'])
if config['apim_name']:
    print_val('APIM Service', config['apim_name'])
print_val('Location', config['location'])
print_val('Subscription ID', config['subscription_id'])

## 📦 Deploy Observability & Cost Resources

Deploy the Bicep template to create:
- Application Insights
- Log Analytics Workspace
- Storage Account (for cost exports)
- Diagnostic Settings (APIM → Application Insights & Log Analytics)

In [ ]:
print_info('Deploying observability and cost management resources...')

# Define deployment name with index (default used by utils function is just the folder name)
deployment_name = config['sample_folder']  # utils uses sample_name as deployment name

# Check if sample deployment already exists
existing_deployment = az.run(
    f'az deployment group show --resource-group {config["rg_name"]} --name {deployment_name} -o json',
    log_command=False
)

if existing_deployment.success:
    print_warning(f'Deployment "{deployment_name}" already exists')
    print_info('This will update the existing deployment with current parameters')
    print_info('Tip: Use sample_index to differentiate multiple sample deployments')

# Get APIM service name if not already set
if not config['apim_name']:
    apim_list_result = az.run(
        f'az apim list --resource-group {config["rg_name"]} -o json',
        log_command=False
    )
    if apim_list_result.success and apim_list_result.json_data and len(apim_list_result.json_data) > 0:
        config['apim_name'] = apim_list_result.json_data[0]['name']
        print_val('Found APIM Service', config['apim_name'])
    else:
        print_error('No APIM service found in resource group. Please deploy infrastructure first.')
        raise SystemExit(1)

# Deploy using the standard utility function with indexed naming
deployment_result = utils.create_bicep_deployment_group_for_sample(
    config['sample_folder'],
    config['rg_name'],
    config['location'],
    {
        'apimServiceName': {'value': config['apim_name']},
        'location': {'value': config['location']},
        'costExportFrequency': {'value': config['cost_export_frequency']},
        'sampleIndex': {'value': config['sample_index']}
    }
)

if not deployment_result.success:
    print_error('Deployment failed')
    raise SystemExit(1)

# Extract outputs
config['app_insights_name'] = deployment_result.get('applicationInsightsName')
config['log_analytics_name'] = deployment_result.get('logAnalyticsWorkspaceName')
config['storage_account_name'] = deployment_result.get('storageAccountName')
config['app_insights_connection_string'] = deployment_result.get('applicationInsightsConnectionString')
config['workbook_name'] = deployment_result.get('workbookName')
config['workbook_id'] = deployment_result.get('workbookId')

# Query for cost export (subscription-scoped resource)
config['cost_export_name'] = f"apim-cost-export-{config['sample_index']}-{config['rg_name']}"

print_ok('Resources deployed successfully')
print_val('Deployment Name', deployment_name)
print_val('Application Insights', config['app_insights_name'])
print_val('Log Analytics Workspace', config['log_analytics_name'])
print_val('Storage Account', config['storage_account_name'])
if config.get('workbook_name'):
    print_val('Azure Monitor Workbook', config['workbook_name'])

## 🔧 Configure Cost Management Export

Automatically set up cost data export to the storage account.

In [ ]:
if not config.get('storage_account_name'):
    print_error('Please run cell 6 (Deploy Observability & Cost Resources) first')
    raise SystemExit(1)

import json
import tempfile
from datetime import datetime, timedelta, timezone

print_info('Configuring automated Cost Management export (managed identity)...')

# Get storage account resource ID
storage_account_id = (
    f'/subscriptions/{config["subscription_id"]}'
    f'/resourceGroups/{config["rg_name"]}'
    f'/providers/Microsoft.Storage/storageAccounts/{config["storage_account_name"]}'
)

# Export scope and name
export_scope = f'/subscriptions/{config["subscription_id"]}'
export_name = config['cost_export_name']
api_version = '2025-03-01'

# Register required resource provider
print_info('Registering Microsoft.CostManagementExports resource provider...')
register_result = az.run(
    'az provider register --namespace Microsoft.CostManagementExports --wait',
    log_command=False
)

if register_result.success:
    print_ok('Resource provider registered successfully')

# Check if export already exists (must use 2025-03-01 API for managed identity exports)
existing_export = az.run(
    f'az rest --method GET '
    f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{export_name}'
    f'?api-version={api_version}" -o json',
    log_command=False
)

if existing_export.success:
    print_warning(f'Cost export "{export_name}" already exists')
    print_info('Deleting existing export to recreate with current settings...')
    az.run(
        f'az rest --method DELETE '
        f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{export_name}'
        f'?api-version={api_version}"',
        log_command=False
    )

# Build recurrence settings
recurrence_map = {'Daily': 'Daily', 'Weekly': 'Weekly', 'Monthly': 'Monthly'}
recurrence = recurrence_map.get(config['cost_export_frequency'], 'Daily')

start_date = (datetime.now(timezone.utc) + timedelta(days=1)).strftime('%Y-%m-%dT00:00:00Z')
end_date = (datetime.now(timezone.utc) + timedelta(days=365)).strftime('%Y-%m-%dT00:00:00Z')

# Build the export body with system-assigned managed identity
# Uses 2025-03-01 API which supports identity-based delivery to storage
export_body = {
    'identity': {
        'type': 'systemAssigned'
    },
    'location': 'global',
    'properties': {
        'definition': {
            'type': 'ActualCost',
            'timeframe': 'MonthToDate',
            'dataSet': {
                'granularity': 'Daily'
            }
        },
        'deliveryInfo': {
            'destination': {
                'type': 'AzureBlob',
                'container': 'cost-exports',
                'rootFolderPath': 'apim-costing',
                'resourceId': storage_account_id
            }
        },
        'schedule': {
            'status': 'Active',
            'recurrence': recurrence,
            'recurrencePeriod': {
                'from': start_date,
                'to': end_date
            }
        },
        'format': 'Csv'
    }
}

print_info('Creating cost export with managed identity...')

# Write body to a temp file for cross-platform compatibility
# (inline JSON in --body breaks on Windows/PowerShell due to quote handling)
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as body_file:
    json.dump(export_body, body_file)
    body_file_path = body_file.name

try:
    export_result = az.run(
        f'az rest --method PUT '
        f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{export_name}'
        f'?api-version={api_version}" '
        f'--body @{body_file_path} -o json',
        log_command=False
    )
finally:
    Path(body_file_path).unlink(missing_ok=True)

if export_result and export_result.success:
    print_ok(f'Cost export created: {export_name}')
    print_val('Export frequency', recurrence)
    print_val('Authentication', 'System-assigned managed identity')
    config['cost_export_configured'] = True

    # Extract the managed identity principal ID from the response
    export_data = json.loads(export_result.text)
    principal_id = export_data.get('identity', {}).get('principalId')

    if principal_id:
        print_info('Assigning Storage Blob Data Contributor role to export identity...')

        role_assignment = az.run(
            f'az role assignment create '
            f'--assignee-object-id {principal_id} '
            f'--assignee-principal-type ServicePrincipal '
            f'--role "Storage Blob Data Contributor" '
            f'--scope {storage_account_id}',
            log_command=False
        )

        if role_assignment.success:
            print_ok('Storage Blob Data Contributor role assigned to export identity')
        else:
            print_warning('Could not assign role - you may need to do this manually')
            print_info(f'Principal ID: {principal_id}')
    else:
        print_warning('Could not retrieve export identity principal ID')
        print_info('You may need to assign Storage Blob Data Contributor role manually')

    print_info('Cost data will be exported automatically starting tomorrow')
else:
    print_error('Failed to create cost export')
    if export_result and export_result.text:
        print_warning(f'Error: {export_result.text[:500]}')

    print()
    print_warning('Continuing without cost export - you can configure it manually later')
    config['cost_export_configured'] = False

In [ ]:
# Trigger the first cost export run immediately (instead of waiting for tomorrow's schedule)
if config.get('cost_export_configured'):
    export_scope = f'/subscriptions/{config["subscription_id"]}'
    export_name = config['cost_export_name']
    api_version = '2025-03-01'

    print_info(f'Triggering first cost export run for "{export_name}"...')

    run_result = az.run(
        f'az rest --method POST '
        f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{export_name}'
        f'/run?api-version={api_version}"',
        log_command=False
    )

    if run_result.success:
        print_ok('Cost export run triggered successfully')
        print_info('Data will appear in the storage container within a few minutes')
    else:
        print_warning('Could not trigger export run - it will run on its next scheduled recurrence')
        if run_result and run_result.text:
            print_info(f'Details: {run_result.text[:300]}')
else:
    print_warning('Cost export was not configured - skipping manual run')

## 🏢 Create Sample API & Business Unit Subscriptions

Create a sample API and multiple APIM subscriptions representing different business units, departments, or applications.

In [ ]:
import json as json_module

print_info('Creating sample API for cost tracking...')

# APIM ARM base URL
apim_base_url = (
    f'/subscriptions/{config["subscription_id"]}'
    f'/resourceGroups/{config["rg_name"]}'
    f'/providers/Microsoft.ApiManagement/service/{config["apim_name"]}'
)
apim_api_version = '2024-06-01-preview'

# Create a sample echo API
api_id = 'cost-tracking-api'
api_path = 'cost-demo'

api_body = {
    'properties': {
        'displayName': 'Cost Tracking Demo API',
        'path': api_path,
        'protocols': ['https'],
        'subscriptionRequired': True,
        'serviceUrl': 'https://httpbin.org'
    }
}

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json_module.dump(api_body, f)
    api_body_path = f.name

try:
    az.run(
        f'az rest --method PUT '
        f'--url "{apim_base_url}/apis/{api_id}?api-version={apim_api_version}" '
        f'--body @{api_body_path} -o json',
        log_command=False
    )
finally:
    Path(api_body_path).unlink(missing_ok=True)

# Create an operation
op_body = {
    'properties': {
        'displayName': 'Get Status',
        'method': 'GET',
        'urlTemplate': '/get'
    }
}

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json_module.dump(op_body, f)
    op_body_path = f.name

try:
    az.run(
        f'az rest --method PUT '
        f'--url "{apim_base_url}/apis/{api_id}/operations/get-status?api-version={apim_api_version}" '
        f'--body @{op_body_path} -o json',
        log_command=False
    )
finally:
    Path(op_body_path).unlink(missing_ok=True)

print_ok(f'Created API: {api_id}')

# Create subscriptions for different business units
# request_weight controls traffic distribution (higher = more requests)
business_units = [
    {'name': 'bu-hr', 'display': 'Business Unit - Human Resources', 'request_weight': 1.0},
    {'name': 'bu-finance', 'display': 'Business Unit - Finance', 'request_weight': 2.5},
    {'name': 'bu-marketing', 'display': 'Business Unit - Marketing', 'request_weight': 0.5},
    {'name': 'bu-engineering', 'display': 'Business Unit - Engineering', 'request_weight': 3.0}
]

print_info(f'Creating {len(business_units)} business unit subscriptions...')

config['subscriptions'] = {}

for bu in business_units:
    sub_id = bu['name']

    # Create subscription via ARM REST API
    sub_body = {
        'properties': {
            'displayName': bu['display'],
            'scope': f'/apis/{api_id}',
            'state': 'active'
        }
    }

    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
        json_module.dump(sub_body, f)
        sub_body_path = f.name

    try:
        result = az.run(
            f'az rest --method PUT '
            f'--url "{apim_base_url}/subscriptions/{sub_id}?api-version={apim_api_version}" '
            f'--body @{sub_body_path} -o json',
            log_command=False
        )
    finally:
        Path(sub_body_path).unlink(missing_ok=True)

    # Get subscription keys via listSecrets
    keys_result = az.run(
        f'az rest --method POST '
        f'--url "{apim_base_url}/subscriptions/{sub_id}/listSecrets'
        f'?api-version={apim_api_version}" -o json',
        log_command=False
    )

    primary_key = None
    if keys_result.success and keys_result.json_data:
        primary_key = keys_result.json_data.get('primaryKey')

    config['subscriptions'][sub_id] = {
        'display_name': bu['display'],
        'primary_key': primary_key,
        'request_weight': bu.get('request_weight', 1.0)
    }

    if result.success and primary_key:
        print_ok(f'  Created subscription: {sub_id}')
    elif result.success:
        print_warning(f'  Created subscription {sub_id} but could not retrieve key')
    else:
        print_error(f'  Failed to create subscription: {sub_id}')

print_ok(f'Created {len(config["subscriptions"])} subscriptions')

## 🚀 Generate Sample API Traffic

Generate sample API calls from each business unit subscription to demonstrate cost tracking and allocation.

This will create request logs in Application Insights and Log Analytics that can be used for cost analysis.

In [ ]:
if config['generate_sample_load']:
    import requests

    print_info('Generating sample API traffic...')

    # Get APIM gateway URL
    apim_info_result = az.run(
        f'az apim show '
        f'--resource-group {config["rg_name"]} '
        f'--name {config["apim_name"]} -o json',
        log_command=False
    )

    gateway_url = apim_info_result.json_data['gatewayUrl']
    api_url = f'{gateway_url}/{api_path}/get'

    print_val('API Endpoint', api_url)
    print_info(f'Sending {config["sample_requests_per_subscription"]} requests per subscription...')

    total_requests = 0
    total_success = 0

    base_count = config['sample_requests_per_subscription']

    for subscription_id, sub_info in config['subscriptions'].items():
        bu_request_count = max(1, int(base_count * sub_info.get('request_weight', 1.0)))
        print_info(f'  Testing {subscription_id} ({bu_request_count} requests, weight={sub_info.get("request_weight", 1.0)})...')
        success_count = 0

        for i in range(bu_request_count):
            try:
                response = requests.get(
                    api_url,
                    headers={'Ocp-Apim-Subscription-Key': sub_info['primary_key']},
                    timeout=10
                )
                if response.status_code == 200:
                    success_count += 1
                total_requests += 1
            except Exception as e:
                print_error(f'    Request failed: {str(e)}')

            # Add small delay to avoid overwhelming the API
            if (i + 1) % 10 == 0:
                time.sleep(0.5)

        total_success += success_count
        print_ok(f'    Completed {success_count}/{bu_request_count} requests')

    print_ok(f'Generated {total_success}/{total_requests} successful API calls')
    print_info('Note: It may take 2-5 minutes for logs to appear in Application Insights and Log Analytics')
else:
    print_info('Sample load generation skipped (generate_sample_load = False)')

## 🔍 Verify Log Ingestion

Waits for diagnostic logs to arrive in Log Analytics (auto-retries for up to 10 minutes).

In [ ]:
print_info('Waiting for APIM logs to arrive in Log Analytics...')
print_info('Log ingestion typically takes 2-5 minutes after generating traffic')
print()

log_analytics_workspace_id = config.get('log_analytics_name')

if not log_analytics_workspace_id:
    print_error('Log Analytics workspace name not found in config')
    raise SystemExit(1)

print_val('Workspace', log_analytics_workspace_id)

# Retry until logs arrive or timeout
max_wait_minutes = 10
poll_interval_seconds = 30
max_attempts = (max_wait_minutes * 60) // poll_interval_seconds
logs_found = False

for attempt in range(1, max_attempts + 1):
    query_cmd = (
        f'az monitor log-analytics query '
        f'--workspace {log_analytics_workspace_id} '
        f'--resource-group {config["rg_name"]} '
        f'--analytics-query "ApiManagementGatewayLogs | where ApimSubscriptionId != \'\' | summarize Count = count()" '
        f'-o json'
    )

    result = az.run(query_cmd, log_command=False)

    if result.success and result.json_data:
        tables = result.json_data.get('tables', [])
        if tables and len(tables) > 0:
            rows = tables[0].get('rows', [])
            if rows and len(rows) > 0:
                row_count = int(rows[0][0])
                if row_count > 0:
                    print_ok(f'✓ Found {row_count} log entries with subscription IDs')
                    logs_found = True
                    break

    elapsed = attempt * poll_interval_seconds
    remaining = (max_wait_minutes * 60) - elapsed
    print_info(f'  ⏳ No logs yet... retrying in {poll_interval_seconds}s ({remaining}s remaining)')
    time.sleep(poll_interval_seconds)

if logs_found:
    print_ok('Log ingestion verified - workbook should now display data')
else:
    print_warning(f'Logs did not appear within {max_wait_minutes} minutes')
    print_info('This can happen with newly created workspaces. Tips:')
    print_info('  1. Wait a few more minutes and re-run this cell')
    print_info('  2. Verify diagnostic settings in Azure Portal')
    print_info('  3. Re-run cell 13 to generate more traffic')

## 📊 Cost Analysis & Sample Kusto Queries

### Cost Allocation Model

| Component | Formula |
|---|---|
| **Base Cost** | Monthly platform cost for the APIM SKU (auto-detected or parameterised) |
| **Base Cost Share** | `Base Monthly Cost × (BU Requests ÷ Total Requests)` |
| **Variable Cost** | `BU Requests × (Rate per 1K ÷ 1000)` |
| **Total Allocated** | `Base Cost Share + Variable Cost` |

The next cell auto-detects your APIM SKU and fetches live pricing from the [Azure Retail Prices API](https://learn.microsoft.com/rest/api/cost-management/retail-prices/azure-retail-prices).

In [ ]:
import requests as http_requests

# ------------------------------
#    FETCH LIVE PRICING FROM AZURE RETAIL PRICES API
# ------------------------------

print_info('Fetching live APIM pricing from Azure Retail Prices API...')

# Detect SKU and capacity from the deployed APIM instance
sku_result = az.run(
    f'az apim show '
    f'--resource-group {config["rg_name"]} '
    f'--name {config["apim_name"]} '
    f'--query "{{sku: sku.name, capacity: sku.capacity, location: location}}" -o json',
    log_command=False
)

if sku_result.success and sku_result.json_data:
    apim_sku = sku_result.json_data['sku']       # e.g. 'BasicV2'
    apim_capacity = sku_result.json_data['capacity']
    apim_location = sku_result.json_data['location']
    print_val('APIM SKU', apim_sku)
    print_val('Capacity (units)', apim_capacity)
    print_val('Region', apim_location)
else:
    apim_sku = 'BasicV2'
    apim_capacity = 1
    apim_location = config['location']
    print_warning(f'Could not detect SKU, defaulting to {apim_sku}')

# Map SKU names to Azure Retail Prices API format
sku_api_name_map = {
    'BasicV2': 'Basic v2',
    'StandardV2': 'Standard v2',
    'PremiumV2': 'Premium v2',
    'Basic': 'Basic',
    'Standard': 'Standard',
    'Premium': 'Premium'
}

# Map location display names to ARM region names
location_arm_map = {
    'East US': 'eastus',
    'East US 2': 'eastus2',
    'West US': 'westus',
    'West US 2': 'westus2',
    'West US 3': 'westus3',
    'Central US': 'centralus',
    'North Europe': 'northeurope',
    'West Europe': 'westeurope',
    'UK South': 'uksouth',
    'Southeast Asia': 'southeastasia'
}
arm_region = location_arm_map.get(apim_location, apim_location.lower().replace(' ', ''))

api_sku_name = sku_api_name_map.get(apim_sku, apim_sku)
pricing_url = (
    'https://prices.azure.com/api/retail/prices'
    f"?$filter=serviceName eq 'API Management'"
    f" and skuName eq '{api_sku_name}'"
    f" and priceType eq 'Consumption'"
    f" and armRegionName eq '{arm_region}'"
)

base_monthly_cost = None
per_k_rate = None
included_requests_k = None

try:
    pricing_response = http_requests.get(pricing_url, timeout=15)
    pricing_data = pricing_response.json()

    for item in pricing_data.get('Items', []):
        meter = item.get('meterName', '')
        price = item.get('retailPrice', 0)
        unit = item.get('unitOfMeasure', '')
        tier_min = item.get('tierMinimumUnits', 0)

        # Unit cost (per hour) - primary unit, not secondary
        if 'Unit' in meter and 'Secondary' not in meter and unit == '1 Hour':
            hourly_rate = price
            base_monthly_cost = round(hourly_rate * 730, 2)  # 730 hrs/month

        # Overage call rate (tier with minimum > 0)
        if 'Calls' in meter and tier_min > 0:
            # Price is per 10K calls
            per_10k_rate = price
            per_k_rate = round(per_10k_rate / 10, 6)
            included_requests_k = int(tier_min * 10)  # tier_min is in 10K units

    if base_monthly_cost and per_k_rate:
        print()
        print_ok('Live pricing retrieved from Azure Retail Prices API')
        print_val('Base monthly cost (per unit)', f'${base_monthly_cost}')
        print_val('Total base cost ({} unit(s))'.format(apim_capacity), f'${round(base_monthly_cost * apim_capacity, 2)}')
        print_val('Included requests', f'{included_requests_k:,}K ({included_requests_k // 1000}M)')
        print_val('Overage rate per 1K requests', f'${per_k_rate}')

        # Adjust for capacity
        base_monthly_cost = round(base_monthly_cost * apim_capacity, 2)
    else:
        raise ValueError('Could not parse unit cost or call rate from API response')

except Exception as e:
    print_warning(f'Could not fetch live pricing: {e}')
    print_info('Falling back to default BasicV2 pricing')
    base_monthly_cost = 150.00
    per_k_rate = 0.003
    included_requests_k = 10000
    print_val('Base monthly cost (default)', f'${base_monthly_cost:.2f}')
    print_val('Overage rate per 1K (default)', f'${per_k_rate}')

# Store in config for use by other cells
config['base_monthly_cost'] = base_monthly_cost
config['per_k_rate'] = per_k_rate
config['included_requests_k'] = included_requests_k

# ------------------------------
#    SAMPLE KUSTO QUERIES
# ------------------------------

print()
print_info('Sample Kusto Queries for Log Analytics:')
print_info('These queries use the ApiManagementGatewayLogs table (resource-specific mode).')
print()

queries = {
    'Cost Allocation by Business Unit': f'''
// Split base APIM cost proportionally + add variable per-request cost
// Pricing source: Azure Retail Prices API ({apim_sku}, {arm_region})
let baseCost = {base_monthly_cost};
let perKRate = {per_k_rate};
let logs = ApiManagementGatewayLogs
| where TimeGenerated > ago(30d) and ApimSubscriptionId != '';
let totalRequests = toscalar(logs | summarize count());
logs
| summarize RequestCount = count() by ApimSubscriptionId
| extend UsageShare = round(RequestCount * 100.0 / totalRequests, 2)
| extend BaseCostShare = round(baseCost * RequestCount / totalRequests, 2)
| extend VariableCost = round(RequestCount * perKRate / 1000.0, 2)
| extend TotalAllocatedCost = round(BaseCostShare + VariableCost, 2)
| order by TotalAllocatedCost desc
| project
    ['Business Unit'] = ApimSubscriptionId,
    ['Requests'] = RequestCount,
    ['Usage Share (%)'] = UsageShare,
    ['Base Cost ($)'] = baseCost,
    ['Base Cost Share ($)'] = BaseCostShare,
    ['Variable Cost ($)'] = VariableCost,
    ['Total Allocated ($)'] = TotalAllocatedCost
''',
    'Cost Breakdown per API per Business Unit': f'''
// Drill down: cost allocation at the API level
let baseCost = {base_monthly_cost};
let perKRate = {per_k_rate};
let logs = ApiManagementGatewayLogs
| where TimeGenerated > ago(30d) and ApimSubscriptionId != '';
let totalRequests = toscalar(logs | summarize count());
logs
| summarize RequestCount = count() by ApimSubscriptionId, ApiId
| extend BaseCostShare = round(baseCost * RequestCount / totalRequests, 2)
| extend VariableCost = round(RequestCount * perKRate / 1000.0, 2)
| extend TotalCost = round(BaseCostShare + VariableCost, 2)
| order by TotalCost desc
| project
    ['Business Unit'] = ApimSubscriptionId,
    ['API'] = ApiId,
    ['Requests'] = RequestCount,
    ['Base Share ($)'] = BaseCostShare,
    ['Variable ($)'] = VariableCost,
    ['Total ($)'] = TotalCost
| take 25
''',
    'Request Count by Business Unit': '''
ApiManagementGatewayLogs
| where TimeGenerated > ago(24h) and ApimSubscriptionId != ''
| summarize RequestCount = count() by ApimSubscriptionId
| order by RequestCount desc
| project BusinessUnit = ApimSubscriptionId, RequestCount
''',
    'Success & Error Metrics by Business Unit': '''
ApiManagementGatewayLogs
| where TimeGenerated > ago(24h) and ApimSubscriptionId != ''
| summarize
    TotalRequests = count(),
    SuccessRequests = countif(ResponseCode < 400),
    ClientErrors = countif(ResponseCode >= 400 and ResponseCode < 500),
    ServerErrors = countif(ResponseCode >= 500)
    by ApimSubscriptionId
| extend SuccessRate = round(SuccessRequests * 100.0 / TotalRequests, 2)
| extend ErrorRate = round((ClientErrors + ServerErrors) * 100.0 / TotalRequests, 2)
| project
    BusinessUnit = ApimSubscriptionId,
    TotalRequests,
    SuccessRequests,
    ClientErrors,
    ServerErrors,
    ['Success Rate (%)'] = SuccessRate,
    ['Error Rate (%)'] = ErrorRate
| order by TotalRequests desc
''',
    'Hourly Request Trends': '''
ApiManagementGatewayLogs
| where TimeGenerated > ago(7d) and ApimSubscriptionId != ''
| summarize RequestCount = count() by bin(TimeGenerated, 1h), ApimSubscriptionId
| render timechart
''',
    'Response Code Distribution': '''
ApiManagementGatewayLogs
| where TimeGenerated > ago(24h) and ApimSubscriptionId != ''
| extend ResponseClass = case(
    ResponseCode >= 200 and ResponseCode < 300, "2xx",
    ResponseCode >= 300 and ResponseCode < 400, "3xx",
    ResponseCode >= 400 and ResponseCode < 500, "4xx",
    ResponseCode >= 500, "5xx",
    "Other")
| summarize RequestCount = count() by ApimSubscriptionId, ResponseClass
| order by ApimSubscriptionId, ResponseClass
| project BusinessUnit = ApimSubscriptionId, ResponseClass, RequestCount
'''
}

for query_name, query in queries.items():
    print(f'📈 {query_name}')
    print('─' * 60)
    print(query.strip())
    print()

print_ok('Query templates ready to use in Log Analytics')

## 🔔 Set Up Budget Alerts per Business Unit

Create Azure Monitor scheduled query alerts that fire when a business unit subscription exceeds a configurable request threshold.

Each alert:
- Runs a Kusto query every **5 minutes** against the Log Analytics workspace
- Triggers when a business unit exceeds the threshold in a **1-hour** rolling window
- Sends notifications via an **Action Group** (email)

> 💡 Adjust `alert_threshold` and `alert_email` below to match your requirements.

In [ ]:
# ------------------------------
#    ALERT CONFIGURATION
# ------------------------------

alert_threshold = 1000        # Request count threshold per BU per hour
alert_email = 'sample@abc.com'              # Email for alert notifications (leave empty to skip)
alert_severity = 2            # 0=Critical, 1=Error, 2=Warning, 3=Informational, 4=Verbose

if not alert_email:
    print_warning('No alert_email configured - skipping budget alert setup')
    print_info('Set alert_email above to enable budget alerts per business unit')
else:
    print_info('Setting up budget alerts per business unit subscription...')

    # Get Log Analytics workspace resource ID
    workspace_result = az.run(
        f'az monitor log-analytics workspace show '
        f'--resource-group {config["rg_name"]} '
        f'--workspace-name {config["log_analytics_name"]} '
        f'--query id -o tsv',
        log_command=False
    )
    workspace_id = workspace_result.text.strip()

    # Create an Action Group for alert notifications
    action_group_name = f'ag-apim-cost-alerts-{config["sample_index"]}'
    print_info(f'Creating action group: {action_group_name}...')

    ag_result = az.run(
        f'az monitor action-group create '
        f'--resource-group {config["rg_name"]} '
        f'--name {action_group_name} '
        f'--short-name apimcost '
        f'--action email cost-alert-email {alert_email} '
        f'-o json',
        log_command=False
    )

    if ag_result.success:
        action_group_id = ag_result.json_data.get('id', '')
        print_ok(f'Action group created: {action_group_name}')
    else:
        print_error(f'Failed to create action group: {ag_result.text}')
        action_group_id = None

    if action_group_id:
        # Get the list of business units from config or define them
        bu_list = list(config.get('subscriptions', {}).keys())
        if not bu_list:
            bu_list = ['bu-hr', 'bu-finance', 'bu-marketing', 'bu-engineering']

        print_info(f'Creating alerts for {len(bu_list)} business units (threshold: {alert_threshold} requests/hour)...')

        for bu_name in bu_list:
            alert_name = f'apim-budget-{bu_name}-{config["sample_index"]}'

            # Kusto query: count requests for this specific BU in the last hour
            kusto_query = (
                f'ApiManagementGatewayLogs '
                f'| where TimeGenerated > ago(1h) and ApimSubscriptionId == \'{bu_name}\' '
                f'| summarize RequestCount = count() '
                f'| where RequestCount > {alert_threshold}'
            )

            # Create the scheduled query alert rule
            result = az.run(
                f'az monitor scheduled-query create '
                f'--resource-group {config["rg_name"]} '
                f'--name {alert_name} '
                f'--display-name "APIM Budget Alert: {bu_name}" '
                f'--description "Fires when {bu_name} exceeds {alert_threshold} API requests per hour" '
                f'--scopes {workspace_id} '
                f'--condition "count \'RequestCount\' from \'{kusto_query}\' > 0" '
                f'--condition-query "{kusto_query}" '
                f'--evaluation-frequency 5m '
                f'--window-size 1h '
                f'--severity {alert_severity} '
                f'--action-groups {action_group_id} '
                f'-o json',
                log_command=False
            )

            if result.success:
                print_ok(f'  Alert created: {alert_name}')
            else:
                # Try alternative approach using REST API
                import json as json_module

                alert_body = {
                    'location': config['location'],
                    'properties': {
                        'displayName': f'APIM Budget Alert: {bu_name}',
                        'description': f'Fires when {bu_name} exceeds {alert_threshold} API requests per hour',
                        'severity': alert_severity,
                        'enabled': True,
                        'evaluationFrequency': 'PT5M',
                        'windowSize': 'PT1H',
                        'scopes': [workspace_id],
                        'criteria': {
                            'allOf': [
                                {
                                    'query': kusto_query,
                                    'timeAggregation': 'Count',
                                    'operator': 'GreaterThan',
                                    'threshold': 0,
                                    'failingPeriods': {
                                        'numberOfEvaluationPeriods': 1,
                                        'minFailingPeriodsToAlert': 1
                                    }
                                }
                            ]
                        },
                        'actions': {
                            'actionGroups': [action_group_id]
                        }
                    }
                }

                alert_id = (
                    f'/subscriptions/{config["subscription_id"]}'
                    f'/resourceGroups/{config["rg_name"]}'
                    f'/providers/Microsoft.Insights/scheduledQueryRules/{alert_name}'
                )

                rest_result = az.run(
                    f'az rest --method PUT '
                    f'--uri https://management.azure.com{alert_id}?api-version=2023-03-15-preview '
                    f"--body '{json_module.dumps(alert_body)}'",
                    log_command=False
                )

                if rest_result.success:
                    print_ok(f'  Alert created (via REST): {alert_name}')
                else:
                    print_error(f'  Failed to create alert for {bu_name}: {rest_result.text[:200]}')

        print()
        print_ok('Budget alerts configured!')
        print_val('Action Group', action_group_name)
        print_val('Alert Email', alert_email)
        print_val('Threshold', f'{alert_threshold} requests per hour per BU')
        print_val('Evaluation', 'Every 5 minutes, 1-hour rolling window')

## 🔗 Access Your Resources

Links to access the deployed resources in Azure Portal.

In [ ]:
print_info('📎 Azure Portal Links:')
print()

base_url = 'https://portal.azure.com/#@/resource'
subscription_path = f'/subscriptions/{config["subscription_id"]}'
rg_path = f'{subscription_path}/resourceGroups/{config["rg_name"]}'

resources = [
    ('Application Insights', f'{rg_path}/providers/Microsoft.Insights/components/{config["app_insights_name"]}/overview'),
    ('Log Analytics Workspace', f'{rg_path}/providers/Microsoft.OperationalInsights/workspaces/{config["log_analytics_name"]}/Overview'),
    ('Storage Account', f'{rg_path}/providers/Microsoft.Storage/storageAccounts/{config["storage_account_name"]}/overview'),
    ('APIM Service', f'{rg_path}/providers/Microsoft.ApiManagement/service/{config["apim_name"]}/overview'),
    ('Cost Management + Exports', f'{subscription_path}/costManagement')
]

if config.get('workbook_id'):
    resources.append(('Azure Monitor Workbook', f'{base_url}{config["workbook_id"]}/workbook'))

for name, path in resources:
    print(f'{name}:')
    print(f'{base_url}{path}')
    print()

print_ok('Setup complete!')
print()
print_info('📊 Cost Allocation is now automated:')
print('  ✓ Request logs flowing to Application Insights & Log Analytics')
if config.get('cost_export_configured'):
    print('  ✓ Cost data automatically exported daily to Storage Account')
else:
    print('  ⚠ Cost export not configured - configure manually via Azure Portal')
print('  ✓ Azure Monitor Workbook ready for cost visualization')
print('  ✓ Business unit usage tracked via APIM subscriptions')
print()
print_info('💡 Cost allocation workflow:')
print('  1. APIM logs each API request with subscription info')
print('  2. Azure exports daily cost data to storage')
print('  3. Correlate usage metrics with costs using subscription IDs')
print('  4. Create chargeback reports per business unit')
print('  5. Set up budget alerts based on usage patterns')
print()
print_info('🔍 Next steps:')
print('  • Open Azure Monitor Workbook to view real-time cost dashboard')
print('  • Query Log Analytics for detailed usage analysis')
print('  • Review Cost Management exports (available after first export run)')
print()
print_info('To clean up resources, open and run: clean-up.ipynb')